In [3]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, Conv2DTranspose
from keras.optimizers import SGD, Adam
from keras.layers.convolutional import UpSampling2D
from keras.layers.core import Reshape

Using TensorFlow backend.


In [4]:
from keras import backend as K
K.set_image_dim_ordering('tf')
'''
image_dim_ordering in 'th' mode the channels dimension (the depth) is at index 1 (e.g. 3, 256, 256). " \
"In 'tf' mode is it at index 3 (e.g. 256, 256, 3).
'''

'\nimage_dim_ordering in \'th\' mode the channels dimension (the depth) is at index 1 (e.g. 3, 256, 256). " "In \'tf\' mode is it at index 3 (e.g. 256, 256, 3).\n'

In [5]:
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(256,256, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu', name='block1_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, (3, 3), activation='relu', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu', name='block2_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu', name='block3_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu', name='block3_conv2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu', name='block3_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='block4_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='block4_conv2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='block4_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='block5_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='block5_conv2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='block5_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

model.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 
                   by_name=True)

for layer in model.layers:
    layer.trainable = False

model.add(ZeroPadding2D((2, 2)))
model.add(Conv2DTranspose(256, (5, 5), activation='relu', name='deconv-256'))
model.add(UpSampling2D((3, 3)))
model.add(ZeroPadding2D((2, 2)))
model.add(Conv2DTranspose(128, (5, 5), activation='relu', name='deconv-128'))
model.add(UpSampling2D((2, 2)))
model.add(ZeroPadding2D((2, 2)))
model.add(Conv2DTranspose(64, (3, 3), activation='relu', name='deconv-64'))
model.add(UpSampling2D((2, 2)))
model.add(ZeroPadding2D((2, 2)))
model.add(Conv2DTranspose(32, (3, 3), activation='relu', name='deconv-32'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2DTranspose(16, (3, 3), activation='relu', name='deconv-16'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2DTranspose(8, (3, 3), activation='relu', name='deconv-8'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2DTranspose(4, (3, 3), activation='relu', name='deconv-4'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2DTranspose(3, (3, 3), activation='sigmoid', padding = 'same', name='deconv-2'))

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='mean_squared_error')

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 258, 258, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 258, 258, 64)      0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 130, 130, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
__________

In [8]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# we create two instances with the same arguments
data_gen_args = dict(featurewise_center=False,
                     featurewise_std_normalization=False,
                     rotation_range=90.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2,
                     data_format = 'channels_last')
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
#image_datagen.fit(images, augment=True, seed=seed)
#mask_datagen.fit(masks, augment=True, seed=seed)

image_generator = image_datagen.flow_from_directory(
    '../../dataset/5.output/images_224x224',
    class_mode=None,
    seed=seed)

mask_generator = mask_datagen.flow_from_directory(
    '../../dataset/5.output/masks_224x224',
    class_mode=None,
    seed=seed)

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)

model.fit_generator(
    train_generator,
    steps_per_epoch=1,
    epochs=1)

Found 120 images belonging to 1 classes.
Found 120 images belonging to 1 classes.
Epoch 1/1
1/1 [==============================] - 323s 323s/step - loss: 11595.3418
